In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib

from glob import glob
import functools
import json
import re

sns.set_theme(style="darkgrid", context="notebook", palette=sns.color_palette("rocket", 4))
matplotlib.rcParams['figure.figsize'] = (20, 100)

%matplotlib inline

In [ ]:
# load all results
results_files = sorted(glob(f"results/*.json"))
all_results = []
# store layers for each model here
layers = {}
for file in results_files:
    # read json
    with open(file, "r") as f:
        results = json.load(f)
    # extract the model name from the filename
    model = re.search(r"results/(.*?).json", file).group(1)
    # save layers without breaking df
    layers[model] = results.pop("layers")
    for task_name, problem in results.items():
        # add the data to the results (to be turned into df)
        all_results.append({
            "task": task_name,
            "model": model,
            "model/task": f"{model}/{task_name}"}
            |
            {
                # keep attributes as arrays
                problem_name: values
                for problem_name, values in problem.items()
            }
        )
# there is no model called "gptr2-nano" so save layers separately
df = pd.DataFrame(all_results)
df.head()

In [ ]:
# split up df into different groups
resnet_df = df[df["model"].str.match(r"resnet\d+$")]
# resort the resnet df (was alphabetical)
# resnet_df = resnet_df.iloc[[2,3,4,0,1]]
base_df = df[df["model"].str.match(r"[a-z0-9]+-base")]
large_df = df[df["model"].str.match(r"[a-z0-9]+-large")]

In [ ]:
# generic compose function
compose = lambda *F: functools.reduce(lambda f, g: lambda x: f(g(x)), F)

In [ ]:
def plot_layer_chart(results: pd.DataFrame, column: str, title: str = None, split="task", transform=None):
    """plot a generic chart with attributes changing across layesr"""
    # plot len(models) plots with len(splits) different attributes (can vary per model)
    splits = results[split].unique()
    models = results["model"].unique()
    fig, axes = plt.subplots(len(models))
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    data = []
    for pivot in splits:
        for model in models:
            # calculate a sub-df for each split-model combination
            sub_df = results[(results[split] == pivot) & (results["model"] == model)]
            # convert into a format seaborn likes
            for i, row in sub_df.iterrows():
                values = row[column]
                for layer, value in enumerate(values):
                    data.append({
                        split: pivot,
                        "model": model,
                        "layer": layer,
                        column: value
                    })
    # convert results to a dataframe
    data = pd.DataFrame(data)
    previous_tasks = None

    # plot results on each axis
    for i, model in enumerate(models):
        ax = axes[i] if len(models) > 1 else axes
        # plot a bar chart
        sns.barplot(data[data["model"] == model], x="layer", y=column, hue=split, ax=ax)
        tasks = data.loc[data["model"] == model,split].unique()
        # only put legend on first plot of each type
        if set(tasks) == previous_tasks:
            ax.get_legend().remove()
        else:
            previous_tasks = set(tasks)
        # perform a transform to the axis
        if transform is not None:
            transform(ax)
        # add vertical lines for resnets
        if not all(layer[1] == 0 for layer in layers[model]):
            for i, layer in enumerate(layers[model]):
                # put the line between layers
                if layer[1] == 0:
                    ax.axvline(i - 0.5, ls="--", color=sns.color_palette()[-2])
        ax.set_title(model)
        ax.set_xlabel("")
    plt.suptitle(title)


In [ ]:
def plot_distribution_chart(results: pd.DataFrame, column: str, title: str = None, split="task", transform=None):
    """plot a generic chart with attributes changing across layesr"""
    # plot len(models) plots with len(splits) different attributes (can vary per model)
    splits = results[split].unique()
    models = results["model"].unique()
    fig, axes = plt.subplots(sum(len(layers[model]) for model in models))
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    data = []
    for pivot in splits:
        for model in models:
            # calculate a sub-df for each split-model combination
            sub_df = results[(results[split] == pivot) & (results["model"] == model)]
            # convert into a format seaborn likes
            for i, row in sub_df.iterrows():
                values = row[column]
                for layer, value in enumerate(values):
                    for neuron, activation in enumerate(value):
                        data.append({
                            split: pivot,
                            "model": model,
                            "layer": layer,
                            column: activation,
                            "neuron": neuron,
                        })
    # convert results to a dataframe
    data = pd.DataFrame(data)
    previous_tasks = None
    idx = 0

    # plot results on each axis
    for i, model in enumerate(models):
        for j, layer in enumerate(layers[model]):
            ax = axes[idx]
            idx += 1
            # plot a bar chart
            model_layer = (data["model"] == model) & (data["layer"] == j)
            sns.lineplot(data[model_layer], x="neuron", y=column, hue=split, ax=ax)
            tasks = data.loc[model_layer,split].unique()
            # only put legend on first plot of each type
            if set(tasks) == previous_tasks:
                ax.get_legend().remove()
            else:
                previous_tasks = set(tasks)
            # perform a transform to the axis
            if transform is not None:
                transform(ax)
            ax.set_title(f"{model}-{layer}")
            ax.set_xlabel("")
    plt.suptitle(title)

In [ ]:
def add_values(ax: plt.Axes) -> plt.Axes:
    """add values to each bar"""
    # calculate the maximum height bar for reference
    max_height = max([bar.get_height() for bar in ax.patches])
    for bar, line in zip(ax.patches, ax.lines):
        x = bar.get_x()
        width = bar.get_width()
        height = bar.get_height()
        y = max(line.get_ydata())
        # plot at the top of an error bar (if there is one) or bar
        if np.isnan(y):
            y = bar.get_height()
        # plot with scaled font size and approximation of height shown
        ax.text(x + width / 2., y + max_height / 50, f"{height:.{int(height < 10)}f}", ha="center", va="bottom", size=min(int(500 / len(ax.patches)), 12))
    return ax

In [ ]:
def draw_horizontal_line(ax: plt.Axes, y: float) -> plt.Axes:
    """draw of horizontal line of a given height"""
    ax.axhline(y=y, color=sns.color_palette()[-1], linestyle="--", linewidth=1)
    return ax

In [ ]:
plot_distribution_chart(df, title="Distributions", column="distribution")